In [ ]:
import concurrent.futures
from functools import partial
from glob import glob
import os

import matplotlib.pyplot as plt
import numpy as np
from obspy import Stream
from pysep.utils.io import read_sem
from scipy.signal import hilbert as analytic

In [ ]:
write = False

In [ ]:
map = 'Brown'
database_type = 'SymGroups'
#database_type = 't20'

In [ ]:
if database_type == 'SymGroups':
    #models = ['ISO', 'XISO']
    models = ['MONO', 'TRIV']    # choose only two model types
elif database_type == 't20':
    models = ['t80', 't100']    # choose only two model types

database_loc = f'/scratch/agupta7/specfem/rectangular_grid/{map}_{database_type}' 

source_file = f'{database_loc}/{models[0]}/OUTPUT_FILES/CMTSOLUTION'
stations_file = f'{database_loc}/{models[0]}/OUTPUT_FILES/STATIONS_FILTERED'

time_length_s = 40 # corresponds to -1s to 39s of data

In [ ]:
st_list = []
workers = os.cpu_count()
read_sem_new = partial(read_sem, source=source_file, stations=stations_file)

for model in models:   
    print(f'loading data for model: {model}')
    
    fids = []
    for fid in glob(f'{database_loc}/{model}/OUTPUT_FILES/seismograms/*R*'):
        fids.append(fid)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
        data = list(executor.map(read_sem_new, fids))
    
    st = Stream()
    for st_data in data:
        st += st_data
    st_list.append(st)

In [ ]:
for st in st_list:
    for tr in st:
        t1 = tr.stats.starttime
        t2 = t1 + time_length_s
        tr.trim(t1,t2)
        
st1_all = st_list[0].copy().differentiate()
st2_all = st_list[1].copy().differentiate()

In [ ]:
# misfit function definitions
# https://github.com/adjtomo/seisflows (commit 6afdd56)

def waveform(syn, obs, nt, dt, *args, **kwargs):
    """
    Direct waveform differencing

    :type syn: np.array
    :param syn: synthetic data array
    :type obs: np.array
    :param obs: observed data array
    :type nt: int
    :param nt: number of time steps in the data array
    :type dt: float
    :param dt: time step in sec
    """
    wrsd = syn - obs

    return np.sqrt(np.sum(wrsd * wrsd * dt))


def envelope(syn, obs, nt, dt, *args, **kwargs):
    """
    Waveform envelope difference from Yuan et al. 2015 Eq. 9

    :type syn: np.array
    :param syn: synthetic data array
    :type obs: np.array
    :param obs: observed data array
    :type nt: int
    :param nt: number of time steps in the data array
    :type dt: float
    :param dt: time step in sec
    """
    env_syn = abs(analytic(syn))
    env_obs = abs(analytic(obs))

    # Residual of envelopes
    env_rsd = env_syn - env_obs

    return np.sqrt(np.sum(env_rsd * env_rsd * dt))

def instantaneous_phase(syn, obs, nt, dt, *args, **kwargs):
    """
    Instantaneous phase difference from Bozdag et al. 2011

    :type syn: np.array
    :param syn: synthetic data array
    :type obs: np.array
    :param obs: observed data array
    :type nt: int
    :param nt: number of time steps in the data array
    :type dt: float
    :param dt: time step in sec
    """
    r = np.real(analytic(syn))
    i = np.imag(analytic(syn))
    phi_syn = np.arctan2(i, r)

    r = np.real(analytic(obs))
    i = np.imag(analytic(obs))
    phi_obs = np.arctan2(i, r)

    phi_rsd = phi_syn - phi_obs

    return np.sqrt(np.sum(phi_rsd * phi_rsd * dt))

def traveltime(syn, obs, nt, dt, *args, **kwargs):
    """
    Cross-correlation traveltime 

    :type syn: np.array
    :param syn: synthetic data array
    :type obs: np.array
    :param obs: observed data array
    :type nt: int
    :param nt: number of time steps in the data array
    :type dt: float
    :param dt: time step in sec
    """
    cc = abs(np.convolve(obs, np.flipud(syn)))

    return (np.argmax(cc) - nt + 1) * dt

In [ ]:
stations_info = np.genfromtxt(stations_file, dtype=str)

stations = stations_info[:,0]
latitudes = np.array([float(i) for i in stations_info[:,2]])
longitudes = np.array([float(i) for i in stations_info[:,3]])

mask = np.char.startswith(stations, 'C')

stations = stations[~mask]
latitudes = latitudes[~mask]
longitudes = longitudes[~mask]

misfit_functions = ['waveform', 'envelope', 'instantaneous_phase', 'traveltime']
n_misfit_functions = len(misfit_functions)
m = np.zeros((n_misfit_functions,len(stations)))

for i, station in enumerate(stations):
    
    st1 = st1_all.select(station=station)
    st2 = st2_all.select(station=station)
    
    for component in ['Z','Y','X']:
    #for component in ['Z']:
        v1 = st1.select(component=component)[0].data
        v2 = st2.select(component=component)[0].data
        
        obs = v1
        syn = v2
        nt = st1[0].stats.npts
        dt = st1[0].stats.delta
        
        for j in range(n_misfit_functions): 
            if misfit_functions[j]=='waveform':
                m[j,i] += waveform(syn, obs, nt, dt)
            elif misfit_functions[j]=='envelope':
                m[j,i] += envelope(syn, obs, nt, dt)
            elif misfit_functions[j]=='instantaneous_phase':
                m[j,i] += instantaneous_phase(syn, obs, nt, dt)
            elif misfit_functions[j]=='traveltime':
                m[j,i] += traveltime(syn, obs, nt, dt)

In [ ]:
if write:
    for i in range(n_misfit_functions):
        np.save(f'{misfit_functions[i]}', [m[i,:], latitudes, longitudes])

In [ ]:
for i in range(n_misfit_functions):
    plt.figure(figsize=(10,10))
    plt.scatter(longitudes, latitudes, c=m[i], s=30, cmap='jet')
    plt.colorbar()
    plt.axis('equal')
    if write: plt.savefig(f'variability_scatter_{models[0]}_vs_{models[1]}_'
                          f'{misfit_functions[i]}_misfit.png', bbox_inches='tight')
    plt.show()

In [ ]:
indices = np.argsort(m[1,:])[::-1]

sorted_stations = stations[indices]
sorted_latitudes = latitudes[indices]
sorted_longitudes = longitudes[indices]

In [ ]:
components = ['Z','Y','X']
vp = np.sqrt(118.3333E9/3378)
vs = np.sqrt(41.4333E9/3378)

for station in sorted_stations[:1]:

    st1 = st1_all.select(station=station)
    st2 = st2_all.select(station=station)
    
    x1 = st1[0].stats.sac['stlo']
    y1 = st1[0].stats.sac['stla']
    x2 = st1[0].stats.sac['evlo']
    y2 = st1[0].stats.sac['evla']
    
    evdp = 75000 # should be read in from the sac header
    
    source_station_distance = np.sqrt( (y2-y1)**2 + (x2-x1)**2 + evdp**2 )
    
    tp = source_station_distance / vp
    ts = source_station_distance / vs
    
    fig, axs = plt.subplots(3,1,figsize=(10,20)) 
    max_velocity = 0
    for i, component in enumerate(components):
        tr1 = st1.select(component=component)[0]
        tr2 = st2.select(component=component)[0]
        axs[i].plot(tr1.times()+tr1.stats.sac['b'], tr1.data, c='k', 
                    label=f'{models[0]}')
        axs[i].plot(tr2.times()+tr2.stats.sac['b'], tr2.data, c='r', 
                    label=f'{models[1]}') 
        axs[i].axvline(x=tp, color='b', linestyle='--')
        axs[i].axvline(x=ts, color='b', linestyle='--')
        axs[i].set_title(f'component = {component}')
        axs[i].set_xlabel('time (s)')
        axs[i].set_ylabel('velocity (m/s)')
        axs[i].legend()
        max_velocity = max([max_velocity, max(abs(tr1.data)), max(abs(tr2.data))])
        
    for i, _ in enumerate(components):
        axs[i].set_ylim([-1.1 * max_velocity, 1.1 * max_velocity])
        
    fig.suptitle(f'{station}')
    
    if write: plt.savefig(f'max_variability_seismograms_{database_type}_'
                          + f'{model[0]}_{model[1]}.png', bbox_inches='tight')
        
    plt.show()